In [1]:
import json
import requests as re
from datetime import datetime

import numpy as np
import pandas as pd

from dimod import ExactCQMSolver, ConstrainedQuadraticModel, QuadraticModel, Integer, Binary
from dwave.system import LeapHybridCQMSampler

In [2]:
difficulty = 'easy'
solver = 'LHCQM' #'ECQM'
time = None

with open('tokens.json') as json_file:
    token = json.load(json_file)['dwave_token']


In [3]:
with re.Session() as s:
    response = s.get('https://sugoku.herokuapp.com/board', params={'difficulty':difficulty})

    solution = solve_sudoku()

    validate = s.post('https://sugoku.herokuapp.com/validate', response.json())


NameError: name 'solve_sudoku' is not defined

In [92]:
validate.json()

{'status': 'unsolved'}

In [ ]:
now = str(datetime.now()).replace(' ','_')

In [87]:
y = 3**2
print(y**3*3*(y-1))

17496


In [92]:
def solve_sudoku(board, solver, token, time):
    s = len(board)
    rule_count = int(2*(s-1)+(np.sqrt(s)-1)**2)

    '''
    b = int(np.sqrt(s))
    boxes = [(x, y) for x in range(b) for y in range(b)]
    '''

    cqm = ConstrainedQuadraticModel()

    field_bin = [[[Binary(f'field_{i}_{j}_val_{k}')for k in range(1, s+1)] for j in range(s)] for i in range(s)]
    
    for field_row in field_bin:
        for field in field_row:
            cqm.add_constraint(sum(field) == 1)

    rules_bin = [[[[[Binary(f'field_{i}_{j}_val_{k}_rule_{l}') for l in range(rule_count)] for k in range(1, s+1)] for j in range(s)] for i in range(s)]]

    # rules restrictions

    '''
    for i in range(s): # preset values must be correct
        for j in range(s):
            if board[i][j] != 0:
                cqm.add_constraint(fields[i][j] == board[i][j])
    
    for row in np.array(fields): # row sum must be correct
        cqm.add_constraint(sum(row) == sum(range(s+1)))

    for col in np.array(fields)[:]: # column sum must be correct
        cqm.add_constraint(sum(col) == sum(range(s+1)))
    
    for i in range(b):  # boxes must be correct
        for j in range(b):
            cqm.add_constraint(sum([fields[i*b+box[0]][j*b+box[1]] for box in boxes]) == sum(range(s+1)))
    
    test = Binary()
    cqm.add_constraint(test*(fields[0][0]-fields[0][1]) >= test)
    
    for i in range(s):
        for j in range(s):
            for x in range(s):
                if x != j:
                    cqm.add_constraint(test*(fields[i][j]-fields[i][x])**2 >= 1)
                
                if x != i:
                    cqm.add_constraint(test*(fields[i][j]-fields[x][j])**2 >= 1)
                
    '''
    '''
    cqm.set_objective(sum(list(np.array(fields).flatten())))
    cqm.substitute_self_loops()
    
    sampler = LeapHybridCQMSampler(token=token)

    sampleset = sampler.sample_cqm(cqm).filter(lambda row: row.is_feasible)

    best = sampleset.filter(lambda row: row.is_feasible == True).first

    result = np.zeros((s, s), dtype=int)
    for i in range(s):
        for j in range(s):
            result[i][j] = best.sample[f'field_{i}_{j}']
    '''
    return rules_bin #result.tolist()
    

In [93]:
test = solve_sudoku(response.json()['board'], solver, token, time)

In [94]:
test

[[[[[BinaryQuadraticModel({'field_0_0_val_1_rule_0': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_1': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_2': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_3': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_4': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_5': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_6': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_7': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_8': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_9': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_10': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_11': 1.0}, {}, 0.0, 'BINARY'),
     BinaryQuadraticModel({'field_0_0_val_1_rule_1

In [195]:
response.json()['board']

[[4, 9, 6, 0, 0, 0, 0, 0, 3],
 [1, 2, 0, 4, 0, 0, 0, 0, 9],
 [5, 0, 0, 0, 0, 0, 1, 0, 6],
 [0, 0, 4, 3, 0, 0, 7, 9, 0],
 [0, 5, 0, 0, 9, 8, 0, 6, 0],
 [0, 8, 0, 7, 0, 4, 3, 0, 5],
 [7, 3, 1, 6, 0, 2, 9, 0, 0],
 [8, 0, 0, 0, 0, 0, 0, 2, 0],
 [9, 0, 2, 5, 4, 0, 0, 0, 7]]